# ML

In [ ]:
newdir = os.path.join(gv.dir_fig, 'ML' )
if not os.path.exists(newdir):
    os.mkdir( newdir )
newdir_outer = newdir

import shutil
dirname_suffix = ''

for rawname_ in rawnames:
    subj,medcond,task = utils.getParamsFromRawname(rawname_)
    for prefix in prefixes:
    #    rawname_ = 'S04_off_move'


        #prefix = 'all'
        regex = '{}_{}.*{}_ML.*.pdf'.format(subj,medcond,prefix)
        fnfound = utsne.findByPrefix(gv.dir_fig, None, prefix, regex=regex)
        print(regex,len(fnfound))
        if len(fnfound):
            newdir = os.path.join(newdir_outer, '{}_{}_{}'.format(subj, medcond, dirname_suffix) )

            if not os.path.exists(newdir):
                os.mkdir( newdir )

            for fn in fnfound:
                fnfull_old = os.path.join(gv.dir_fig,fn)
                fnfull = os.path.join(newdir,fn)
                shutil.move(fnfull_old,fnfull)

#    fnfound

## Strip heavy

In [ ]:
%load_ext autoreload
%autoreload 2
from os.path import join as pjoin

from time import time
import zipfile
import utils_postprocess as pp

import utils_postprocess_HPC as postp

recent = postp.listRecent(2)
#%debug

bad = []
for fni,fn in list( enumerate(recent) ):# [84:]:
    if fn.startswith('_!'):
        continue
    fn_full = pjoin(gv.data_dir, fn)
    try:
        t0= time()
        f = np.load(fn_full,allow_pickle=True )
        res_cur = pp.extractLightInfo(f)
        f.close()
        del f
        fn_new = '_!' + fn
        fn_new_full = pjoin(gv.data_dir, fn_new)


        gc.collect()
        t1 = time()

        np.savez(fn_new_full,results_light=res_cur)
        oldsz = os.path.getsize(fn_full) / 1024 **3
        newsz = os.path.getsize(fn_new_full) / 1024 **3
        print(fn)
        print(f'  old sz = {oldsz:.4f} Gb, new sz = {newsz:.4f} Gb, took {t1-t0:.2f}s')
    except zipfile.BadZipFile as e:
        print(f'{fn} is corrupted: {e}')
        bad += [fn]

#print ( pp.total_size(f) /  pp.total_size(res_cur) )

# Nonlin Proj

In [ ]:
newdir = os.path.join(gv.dir_fig, 'tSNE-like' )
if not os.path.exists(newdir):
    os.mkdir( newdir )
newdir_outer = newdir

import shutil
subskip_val = 1
dirname_suffix = 'subskip{}'.format(subskip_val)

for rawname_ in rawnames:
    subj,medcond,task = utils.getParamsFromRawname(rawname_)
    for prefix in prefixes:
    #    rawname_ = 'S04_off_move'


        #prefix = 'all'
        regex = '{}_{}.*{}_tSNE_.*_subskip{}.pdf'.format(subj,medcond,prefix,subskip_val)
        fnfound = utsne.findByPrefix(gv.dir_fig, None, prefix, regex=regex)
        print(regex,len(fnfound))
        if len(fnfound):
            newdir = os.path.join(newdir_outer, '{}_{}_{}'.format(subj, medcond, dirname_suffix) )

            if not os.path.exists(newdir):
                os.mkdir( newdir )

            for fn in fnfound:
                fnfull_old = os.path.join(gv.dir_fig,fn)
                fnfull = os.path.join(newdir,fn)
                shutil.move(fnfull_old,fnfull)

#    fnfound

# move files

In [ ]:
# %load ../helper_scripts/move_between_out_folders.py
import os
from datetime import datetime 
import shutil

subdir_from = 'joint_noskip'
subdir_to = 'per_subj_best_LFP'
#subdir_from = subdir_to

data_dir = os.path.expandvars('$DATA_DUSS')

start_year,start_month,start_day,start_hour,start_minute = 2021,9,20,19,0
#year,month,day,hour,minute = 2021,9,20,19,0


dirpath = os.path.join(data_dir,subdir_from )
#ls = os.listdir(dirpath )


dirpath_to = os.path.join(data_dir,subdir_to)

if not os.path.exists(dirpath_to):
    print(f'create {dirpath_to}')
    os.mkdir(dirpath_to)

from pathlib import Path
ls = sorted(Path(dirpath).iterdir(), key=os.path.getmtime)

do_move = 1

dates = []
fns = []
dirs = []
fns_true = []
for fn in ls:
    #fn_full = os.path.join(data_dir,subdir_from,fn)
    fn_full = fn
    fn2_full = None
    st = os.stat(fn_full)
    dt = datetime.fromtimestamp(st.st_mtime)

    s = f'{start_year}-{start_month}-{start_day} {start_hour}:{start_minute}:00'
    dt_start = datetime.strptime(s , '%Y-%m-%d %H:%M:%S') 

    if dt < dt_start: #  and dt <= dt_end:
        continue
    #print(f'candiate {fn}, {dt}')
    dates += [dt]
    fns += [fn]

    if os.path.isdir(fn_full):
        dirs += [fn]
    else:
        fns_true += [fn]

    if not fn.is_dir():
        if fn.stem.find('_all_') >= 0:
            print(fn.stem[:40], 'is all')
            #continue

        #newstem = fn.name
        if fn.stem.startswith('__p') or fn.stem.startswith('_!__p'):
            f = np.load(fn,allow_pickle=1)            
            if fn.stem.startswith('__p'):
                rl = f['results_cur'][()]
                rnstr = rl['pars']['rawnames']
                subj = rnstr[:3]
                newstem = '_' + subj + fn.name[1:]
            if fn.stem.startswith('_!__p'):
                rl = f['results_light'][()]
                rnstr = rl['pars']['rawnames']
                subj = rnstr[:3]
                newstem = '_!_' + subj + fn.name[3:]

            fn2_full = os.path.join(dirpath_to,newstem)
            if do_move:
                shutil.move(fn_full, fn2_full)  
        # well, I should only move completed.. so I may need to use indtool info, which is painful
        #shutil.move(fn_full, fn2_full)  
        #print(fn_full,'   ->\n    ',fn2_full)
        print(fn.stem[:40], newstem[:40])

        #break

sdts = list(sorted(dates) ) 
print( f'Earliest {dates[0] } : {fns[0]},\n Latest {dates[-1] } {fns[-1]}')
                                               


In [ ]:
import gc; gc.collect()

In [ ]:
fn_full

In [ ]:
fn2_full

In [ ]:
list

In [ ]:
list( rl.keys() )

In [ ]:
list(f.keys())

In [ ]:
dirpath

In [ ]:
help(os.m)

In [ ]:
dirs[0],dirs[-1]

In [ ]:
fns_true